In [1]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
from pathlib import Path
import pandas as pd
import requests
import random
import time
import os
import re

BASE_DIR = Path().resolve()
CHROME_DRIVEVR = BASE_DIR / "chromedriver-win64" / "chromedriver.exe"
WAIT_TIME = 15

In [2]:
service = Service(executable_path=CHROME_DRIVEVR)

options = Options()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.google.com")

def wait_for_element(driver, by, element_identifier, timeout=WAIT_TIME, all_elements=False):
  try:
    element_present = EC.presence_of_element_located((by, element_identifier))
    WebDriverWait(driver, timeout).until(element_present)
  except Exception as e:
    return None
  if not all_elements:
    return driver.find_element(by, element_identifier)
  else:
    return driver.find_elements(by, element_identifier)


In [3]:
google_search_field = wait_for_element(driver, By.CSS_SELECTOR, "textarea[title='Search']")

google_search_field.clear()
google_search_field.send_keys("car dealers in new zealand")
google_search_field.send_keys(Keys.ENTER)

places = wait_for_element(driver, By.XPATH, "//div[span[text()='More places']]")
places.click()

time.sleep(2)

decimal_pattern = r'\b\d\.\d\b'
integer_pattern = r'\(\d+\)'
text_pattern = r'\b[a-zA-Z][a-zA-Z\s]*[a-zA-Z]\b'
phone_pattern = r'\+\d(?:\s*\d)*\d'
time_pattern = r'\b\d{1,2}:\d{2}\b'


In [4]:
divs = wait_for_element(driver, By.CLASS_NAME, "cXedhc", all_elements=True)
results_container = driver.find_element(By.ID, "res")
data = []

for div in divs:
    
    name = div.find_element(By.CSS_SELECTOR, "div[role='heading']").text
    meta_div = div.find_element(By.XPATH, ".//div[@role='heading']/following-sibling::div[1]").text
    rating_stars = re.findall(decimal_pattern, meta_div)[0]
    reviews = re.findall(integer_pattern, meta_div)[0].replace('(', '').replace(')', '')
    category = re.findall(text_pattern, meta_div)[0]
    location = div.find_element(By.XPATH, ".//div[@role='heading']/following-sibling::div[2]").text
    phone_div = div.find_element(By.XPATH, ".//div[@role='heading']/following-sibling::div[3]").text
    contact_number = re.findall(phone_pattern, phone_div)[0]

    div.click()
    time.sleep(2)
    
    website = wait_for_element(driver, By.LINK_TEXT, "Website").get_attribute("href")
    description = wait_for_element(driver, By.XPATH, "//div[@class='PQbOE']/following-sibling::div[@data-long-text]", timeout=2)
    
    if not description:
        description = "No Description Available"
    else:
        description = description.text
        
    data.append( {
        'Name': name,
        'Rating': rating_stars,
        'Reviews': reviews,
        'Category': category,
        'Location': location,
        'Contact Number': contact_number,
        'Website': website,
        'Description': description
    } )


In [5]:
df = pd.DataFrame(data)
df

,Name,Rating,Reviews,Category,Location,Contact Number,Website,Description
0,NZC New Zealand Car Auckland,4.8,679,Used car dealer,"Auckland, New Zealand",+64 800 692 800,http://www.nzc.kiwi/,"""New Zealand Car is a rapidly growing, 100% Ki..."
1,AJ Motors Henderson,4.9,697,Used car dealer,"Auckland, New Zealand",+64 9 355 8888,https://www.ajmotors.co.nz/,"""Why Choose AJ Motors? AJ Motors was establish..."
2,City Motor Group,4.8,478,Used car dealer,"Auckland, New Zealand",+64 800 333 803,http://www.citymotorgroup.co.nz/,"""City Motor Group has successfully assisted ov..."
3,Budget Car Sales (Manukau),4.7,573,Used car dealer,"Auckland, New Zealand",+64 9 263 1001,https://www.budgetcarsales.co.nz/#utm_source=g...,"""Looking for a used car in Auckland? Budget Ca..."
4,AJ Motors Hamilton,4.9,579,Used car dealer,"Hamilton, New Zealand",+64 7 843 8888,https://linktr.ee/ajmotorshamilton,"""Allow AJ Motors Hamilton to be your one-stop ..."
5,AJ Motors Hornby Christchurch,4.9,533,Used car dealer,"Christchurch, New Zealand",+64 3 341 5322,https://www.ajmotors.co.nz/vehicles?Dealership...,"""AJ Motors Christchurch is your one-stop shop ..."
6,Infinity Cars,4.8,310,Used car dealer,"Auckland, New Zealand",+64 9 443 7100,http://www.infinitycars.co.nz/,No Description Available
7,NZ Motors 4WD'S limited,4.9,409,Car dealer,"Auckland, New Zealand",+64 21 155 7781,https://nzmotorsltd.co.nz/,No Description Available
8,Great Wall Motors Wellington,4.7,54,Used car dealer,"Wellington, New Zealand",+64 4 383 9886,https://www.greatwallmotors.nz/,No Description Available
9,NZC,4.6,203,Used car dealer,"Christchurch, New Zealand",+64 800 778 807,http://www.nzc.kiwi/,"""New Zealand Car is a rapidly growing, 100% Ki..."
